In [1]:
from nltk.corpus import wordnet as wn
import nltk 
import re
import math

In [2]:
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', sentence).strip()

def get_stopwords():
    stopwords_file = open(f"./resources/utils/stop_words__frakes_baeza-yates.txt", "r")
    stopwords_list = []
    for word in stopwords_file:
        stopwords_list.append(word.replace('\n', ''))
    stopwords_file.close()

    stopwords_file = open(f"./resources/utils/stop_words_1.txt", "r")
    stopwords_list = []
    for word in stopwords_file:
        stopwords_list.append(word.replace('\n', ''))
    stopwords_file.close()

    stopwords_file = open(f"./resources/utils/stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords_file:
        stopwords_list.append(word.replace('\n', ''))
    stopwords_file.close()
    
    stopwords_list = list(set(stopwords_list))

    return stopwords_list
     
def remove_stopwords(sentence, stopwords_list):
    return [word for word in sentence.split() if word not in stopwords_list]



### Esercitazione su Word Sense Disambiguation: 

#### INPUT : 
l’input per questa esercitazione è costituito da coppie di termini contenute nel file WordSim353 (disponibile nei
formati .tsv e .csv)
- Il file contiene 353 coppie di termini utilizzati come testset in varie competizioni internazionali
- A ciascuna coppia è attribuito un valore numerico [0,10], che rappresenta la similarità fra gli elementi della coppia.

In [3]:
import csv

sim_file = f"resources/WordSim353/WordSim353.csv"

pairs = []
with open(sim_file, 'r', encoding = "utf-8") as f:
    reader = csv.reader(f)
    reader.__next__()
    for lines in reader:
        pairs.append(lines)
    
print("WordSim353 correctly loaded!")


WordSim353 correctly loaded!


#### Prima parte

Implementare tre misure di similarità basate su WordNet. Per ciascuna delle misure di similarità, calcolare:
- gli indici di correlazione di Spearman e
- gli indici di correlazione di Pearson fra i risultati ottenuti e quelli ‘target’ presenti nel file annotato.

Le misure da implementare sono le seguenti: 
- Wu & Palmer: 
$$cs(s_1, s_2) = \frac{2 \cdot depth(LCS)}{depth(s_1) + depth(s_2)}$$

in cui $LCS$ rappresenta il primo antenato comune (Lowest Common Subsumer) e depth è la funzione che misura la distanza fra la radice di WordNet e il sysnet $x$. 

***L’obiettivo è implementare la misura di similarità di Wu & Palmer! Non dobbiamo fare ciò viene già fatto nell’implementazione di nltk o altre librerie!***



In [4]:
def get_LCS(sysnet1, sysnet2):
    hyperonyms1 = []
    hyperonyms2 = []

    hyperonyms1.append(sysnet1) # il primo elemento della lista è il synset da cui si parte, quindi la foglia
    hyperonyms2.append(sysnet2) # il primo elemento della lista è il synset da cui si parte, quindi la foglia

    # per ogni synset si prendono gli iperonimi e si aggiungono alla lista
    for synset in hyperonyms1:
        synset_hyp = synset.hypernyms()
        #print("\nPER ", str(synset), " GLI IPERONIMI SONO: ", synset_hyp)

        if synset_hyp:
            hyperonyms1.extend(synset_hyp)

    # per ogni synset si prendono gli iperonimi e si aggiungono alla lista
    for synset in hyperonyms2:
        synset_hyp = synset.hypernyms()
        if synset_hyp:
            hyperonyms2.extend(synset_hyp)

    for syn1 in hyperonyms1:
        for syn2 in hyperonyms2:
            # il primo synset che è in comune è il più basso (LCS )
            if syn1 == syn2:
                return syn1 #LCS: Lowest Common Subsumer

    return None

In [5]:
def type_check(list_of_synset):
    for syn in list_of_synset:
        if not isinstance(syn, nltk.corpus.reader.wordnet.Synset):
            raise TypeError("The input parameters must be Synset objects")
    return True

def wu_palmer_similarity(syn1, syn2):
    """
    Returns the Wu-Palmer similarity between synset.
    """

    if isinstance(syn1, nltk.corpus.reader.wordnet.Synset) and isinstance(syn2, nltk.corpus.reader.wordnet.Synset):
        print("\nSYN1: " + str(syn1))
        print("SYN2: " + str(syn2))
        LCS = get_LCS(syn1, syn2)
        print("LCS: " + str(LCS))

        lcs_depth = 0
        
        if LCS:
            lcs_depth = LCS.max_depth()

        # TODO: CAPIRE SE USARE MAX DEPTH O MIN DEPTH
        cs = (2 * (lcs_depth)) / ((syn1.max_depth()) + (syn2.max_depth()))
        
        print("WE PALMER SIMILARITY MANUAL")
        print(cs)

        print("WU PALMER SIMILARITY NLTK")
        print(syn1.wup_similarity(syn2))

        return cs
    else:
        raise TypeError("The input parameters must be Synset objects")
    


    return 0


"""

    oggetto

    veicolo


barca      automobile 


            macchina
"""

'\n\n    oggetto\n\n    veicolo\n\n\nbarca      automobile \n\n\n            macchina\n'

In [6]:
wu_palmer_similarity(wn.synsets("boat"), wn.synsets("car"))

TypeError: The input parameters must be list of Synset objects

- Shortest Path:
$$sim_{path}(s_1, s_2) = 2 \cdot depthMax - len(s_1, s_2)$$

in cui $depthMax$ è un valore fissato per una specifica versione di WordNet. 

In [ ]:
#CALCOLO REALE depthMax = max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())
depth_max = 20

# TODO: CONTROLLARE QUA -> C'E' QUALCOSA DA SISTEMARE
def path_len_between_two_synset(synset1, synset2):
    LCS = get_LCS(synset1, synset2)
    print("LCS: ", LCS)
    print("LCS LIBRARY: ", synset1.lowest_common_hypernyms(synset2))
    if LCS:
        print("synset1.max_depth(): ", synset1.max_depth())
        print("synset2.max_depth(): ", synset2.max_depth())
        print("LCS.max_depth(): ", LCS.max_depth())
        
        distance = (synset1.max_depth() - LCS.max_depth()) + (synset2.max_depth() - LCS.max_depth())
        print("DISTANCE: ", distance)
        print("DISTANCE LIBRARY: ", (synset1.shortest_path_distance(LCS) + synset2.shortest_path_distance(LCS)))

        return distance

def shortest_path(synset1, synset2):
    return 2 * depth_max - path_len_between_two_synset(synset1, synset2)


In [ ]:
shortest_path_value = shortest_path(wn.synsets("dog")[0], wn.synsets("computer")[0])
print(shortest_path_value)

wn.synsets("dog")[0].path_similarity(wn.synsets("cat")[0])

LCS:  Synset('whole.n.02')
LCS LIBRARY:  [Synset('whole.n.02')]
synset1.max_depth():  13
synset2.max_depth():  8
LCS.max_depth():  3
DISTANCE:  15
DISTANCE LIBRARY:  10
25


0.2

- Leakcock & Chodorow: 
$$sim_{LC}(s_1, s_2) = - log \frac{len(s_1, s_2)}{2 \cdot depthMax}$$

In [ ]:
def leakcock_chodorow(synset1, synset2):
    fract = shortest_path(synset1, synset2) / (2 * depth_max)
    return - math.log(fract)

In [ ]:
print("MANUAL: ", leakcock_chodorow(wn.synsets("dog")[0], wn.synsets("computer")[0]))

print("LIBRARY: ", wn.synsets("dog")[0].lch_similarity(wn.synsets("computer")[0]))

LCS:  Synset('whole.n.02')
LCS LIBRARY:  [Synset('whole.n.02')]
synset1.max_depth():  13
synset2.max_depth():  8
LCS.max_depth():  3
DISTANCE:  15
DISTANCE LIBRARY:  10
MANUAL:  0.4700036292457356
LIBRARY:  1.2396908869280152


#### Calcolo della similarità

Le funzioni precedentemente implementate richiedono come input sensi e non i termini. Quindi per calcolare la similarità fra 2 termini è necessario prendere la massima similarità fra tutti i sensi del primo termine e tutti i sensi del secondo termine. 

L'ipotesi quindi è che i due termini funzionino come contesto di disambiguazione l'uno per l'altro. 

L'equazione che formalizza questa idea è la seguente: 

$$sim(w_1, w_2) = \max_{c_1 \in s(w_1), c_2 \in s(w_2)} [sim(c_1, c_2)]$$

***NB!*** Per calcolare gli indici di correlazione non si è interessati a entrare nel merito di come sono calcolati, possiamo prendere delle funzioni prefatte e usarle.

### Seconda parte

Implementare l’algoritmo di Lesk (NON (!=) usare implementazione esistente, e.g., in nltk…).
1. Estrarre 50 frasi dal corpus SemCor (corpus annotato con i synset di
WN) e disambiguare (almeno) un sostantivo per frase. 
Calcolare l’accuratezza del sistema implementato sulla base dei sensi annotati in
SemCor.
    - SemCor è disponibile all’URL http://web.eecs.umich.edu/~mihalcea/downloads.html
2. Randomizzare la selezione delle 50 frasi e la selezione del termine da disambiguare, 
e restituire l’accuratezza media su (per esempio) 10 esecuzioni del programma.

In [ ]:
stop_words = get_stopwords()

In [ ]:
def get_set_of_words(phrase):
    """
    Returns the set of words of a phrase.
    """
    phrase = remove_punctuation(phrase)
    set_of_words = remove_stopwords(phrase, stop_words)
    
    return set_of_words

In [ ]:
def get_context_of_phrase(sentence):
    """
    Returns the context of a phrase.
    """
    sentence = remove_punctuation(sentence)
    set_of_words = remove_stopwords(sentence, stop_words)
    return set_of_words

def compute_overlap(signature, context):
    """
    returns the number of words in common between signature and context
    """

    number_of_words_in_common = len(list(set(signature) & set(context)))

    return number_of_words_in_common

def get_signature(synset):
    """
    returns the signature of synset
    """

    gloss_of_synset = synset.definition() #gloss of synset
    examples_of_synset = synset.examples() #examples of synset

    # print("\ngloss_of_synset")
    # print(gloss_of_synset)

    # print("examples_of_synset")
    # print(examples_of_synset)

    initial_signature = [gloss_of_synset] + examples_of_synset

    signature = []
    for phrase in initial_signature:
        set_of_words = get_set_of_words(phrase)
        signature.append(set_of_words)

    result = sum(signature, []) # flatten list of lists
    return list(set(result)) # remove duplicates

def lesk(word, sentence):
    """
    returns best sense of word
    """

    best_sense = None #most frequent sense for word
    max_overlap = 0
    context = get_context_of_phrase(sentence) #set of words in sentence

    print("CONTEXT:")
    print(context)

    for synset in wn.synsets(word):
        print("\n" + str(synset))
        signature = get_signature(synset) #set of words in the gloss and examples of sense

        print("SIGNATURE:")
        print(signature)

        overlap = compute_overlap(signature, context)
        print("OVERLAP: " + str(overlap) + "\n")
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = synset

    return best_sense

In [ ]:
best_sense = lesk("dog", "I saw a dog in the park")

print("best sense:")
print(best_sense) 

CONTEXT:
['I', 'dog', 'park']

Synset('dog.n.01')
SIGNATURE:
['domesticated', 'prehistoric', 'wolf', 'dog', 'member', 'man', 'times', 'barked', 'Canis', 'occurs', 'descended', 'genus', 'night', 'common', 'breeds']
OVERLAP: 1


Synset('frump.n.01')
SIGNATURE:
['unattractive', 'woman', 'frump', 'dull', 'dog', 'reputation', 'unpleasant', 'real', 'girl']
OVERLAP: 1


Synset('dog.n.03')
SIGNATURE:
['term', 'dog', 'informal', 'lucky', 'man']
OVERLAP: 1


Synset('cad.n.01')
SIGNATURE:
['dirty', 'dog', 'reprehensible', 'morally']
OVERLAP: 1


Synset('frank.n.02')
SIGNATURE:
['bread', 'beef', 'minced', 'pork', 'roll', 'smoked', 'smoothtextured', 'sausage', 'served']
OVERLAP: 0


Synset('pawl.n.01')
SIGNATURE:
['forward', 'moving', 'wheel', 'fits', 'prevent', 'backward', 'ratchet', 'catch', 'hinged', 'move', 'notch']
OVERLAP: 0


Synset('andiron.n.01')
SIGNATURE:
['hot', 'touch', 'supports', 'logs', 'andirons', 'metal', 'fireplace']
OVERLAP: 0


Synset('chase.v.01')
SIGNATURE:
['dog', 'chased', 